In [ ]:
from sympy import symbols, Or, And, Not, satisfiable

# (수정) 1-based 리터럴 사용 예: x1, x2, x3, x4
clauses = [
    (1, -2, 3),   # x1 ∨ ¬x2 ∨ x3
    (-1, 2, 4)    # ¬x1 ∨ x2 ∨ x4
]
num_vars = 4

def solve_3sat(clauses, num_vars):
    # 내부 변수는 x0..x_{n-1} 그대로 유지 (encode 함수와 호환되게)
    vars = symbols(f'x0:{num_vars}')
    expr_clauses = []
    for clause in clauses:
        literals = []
        for lit in clause:
            idx = abs(lit) - 1
            var = vars[idx]
            if lit > 0:
                literals.append(var)
            else:
                literals.append(Not(var))
        expr_clauses.append(Or(*literals))
    sat_expr = And(*expr_clauses)
    models = satisfiable(sat_expr, all_models=True)
    results = []
    for m in models:
        # 기존 encode 함수가 기대하는 형태: {'x0': True, 'x1': False, ...}
        assignment = {f'x{i}': bool(m.get(vars[i], False)) for i in range(num_vars)}
        results.append(assignment)
    return results

In [17]:
# 3-SAT 문제: (x0 ∨ ¬x1 ∨ x2) ∧ (¬x0 ∨ x1 ∨ x3)
clauses = [
    [0, -1, 2],
    [-0, 1, 3]
]
num_vars = 4

solutions = solve_3sat(clauses, num_vars)
for sol in solutions:
    print(sol)

{'x0': False, 'x1': True, 'x2': True, 'x3': False}
{'x0': False, 'x1': False, 'x2': True, 'x3': False}
{'x0': False, 'x1': True, 'x2': False, 'x3': False}
{'x0': False, 'x1': False, 'x2': False, 'x3': False}
{'x0': True, 'x1': True, 'x2': True, 'x3': False}
{'x0': True, 'x1': False, 'x2': True, 'x3': False}
{'x0': True, 'x1': True, 'x2': False, 'x3': False}
{'x0': True, 'x1': False, 'x2': False, 'x3': False}
{'x0': False, 'x1': True, 'x2': True, 'x3': True}
{'x0': False, 'x1': False, 'x2': True, 'x3': True}
{'x0': True, 'x1': True, 'x2': True, 'x3': True}
{'x0': True, 'x1': True, 'x2': False, 'x3': True}


In [ ]:
from itertools import product
from qiskit.quantum_info import SparsePauliOp

def build_hamiltonian_from_solutions(solutions, num_qubits, penalty=1.0):
    solution_tuples = [tuple(bool(sol[f"x{i}"]) for i in range(num_qubits)) for sol in solutions]
    satisfying_set = set(solution_tuples)

    all_assignments = list(product([False, True], repeat=num_qubits))

    paulis = []
    coeffs = []

    for a in all_assignments:
        if a in satisfying_set:
            continue 

        for mask in product([0,1], repeat=num_qubits):
            # mask는 어떤 위치에 Z를 넣을지 결정
            pauli_str = ""
            phase = 1
            for j in range(num_qubits):
                if mask[j] == 1:
                    pauli_str += "Z"
                    if a[j]:
                        phase *= -1
                else:
                    pauli_str += "I"
            coeff = penalty * phase / (2**num_qubits)
            paulis.append(pauli_str)
            coeffs.append(coeff)

    H = SparsePauliOp(paulis, coeffs)
    return H.simplify()

solutions = [
    {'x0': False, 'x1': True, 'x2': True, 'x3': False},
    {'x0': False, 'x1': False, 'x2': True, 'x3': False},
    {'x0': False, 'x1': True, 'x2': False, 'x3': False},
    {'x0': False, 'x1': False, 'x2': False, 'x3': False},
    {'x0': True, 'x1': True, 'x2': True, 'x3': False},
    {'x0': True, 'x1': False, 'x2': True, 'x3': False},
    {'x0': True, 'x1': True, 'x2': False, 'x3': False},
    {'x0': True, 'x1': False, 'x2': False, 'x3': False},
    {'x0': False, 'x1': True, 'x2': True, 'x3': True},
    {'x0': False, 'x1': False, 'x2': True, 'x3': True},
    {'x0': True, 'x1': True, 'x2': True, 'x3': True},
    {'x0': True, 'x1': True, 'x2': False, 'x3': True}
]

H = build_hamiltonian_from_solutions(solutions, num_qubits=4, penalty=1.0)
print(H)


SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIZI', 'ZIZZ', 'ZZII', 'ZZIZ'],
              coeffs=[ 0.25 +0.j, -0.25 +0.j,  0.125+0.j, -0.125+0.j,  0.125+0.j, -0.125+0.j,
  0.125+0.j, -0.125+0.j, -0.125+0.j,  0.125+0.j])


In [23]:
def hamiltonian_to_string_terms(H):
    terms = []
    for pauli, coeff in zip(H.paulis, H.coeffs):
        label = pauli.to_label()
        ops = []
        for i, p in enumerate(label):
            if p == "I":
                continue
            else:
                ops.append(f"{p}{i}")
        if ops:
            term_str = "*".join(ops)
        else:
            term_str = "I" 
        terms.append(f"{coeff.real:.6f} * {term_str}")
    return terms


terms = hamiltonian_to_string_terms(H)
for t in terms:
    print(t)

0.250000 * I
-0.250000 * Z3
0.125000 * Z2
-0.125000 * Z2*Z3
0.125000 * Z1
-0.125000 * Z1*Z3
0.125000 * Z0*Z2
-0.125000 * Z0*Z2*Z3
-0.125000 * Z0*Z1
0.125000 * Z0*Z1*Z3
